In [1]:
# Set up Notebook
% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm


# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [2]:
train_data = pd.read_csv('trainhh.csv')
test_data = pd.read_csv('testhh.csv')
labels = ['logFCS', 'RCSI', 'HDDS']

In [3]:
x_train=train_data.drop(labels, axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_test=test_data.drop(labels, axis=1)
x_test = x_test.drop(x_test.columns[0], axis=1)

x_train.head()

,clust_L12raincytot,clust_L12day1rain,clust_L12maxdays,clust_floodmax,clust_cells_own,clust_price,clust_thinn,clust_roof,clust_hhsize,clust_hh_age,clust_hh_gender,clust_asset,clust_dist_road,clust_dist_admarc,clust_percent_ag,clust_nutri_reten_constrained,clust_elevation,ipc_lag1,ipc_lag12
0,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
1,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
2,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
3,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
4,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN


In [33]:
import numpy as np
rcsi_train= np.arcsinh(y_train[labels[1]])

rcsi_test= np.arcsinh(y_test[labels[1]])

In [34]:
rcsi_train.head()

0    3.402307
1    3.527224
2    0.000000
3    2.776472
4    2.094713
Name: RCSI, dtype: float64

In [32]:
y_train = train_data[labels]
y_test = test_data[labels]
y_train.head()

,logFCS,RCSI,HDDS
0,4.369448,15,6
1,3.433987,17,3
2,3.663562,0,6
3,3.433987,8,4
4,3.597312,4,5


In [23]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [36]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, rcsi_train)

# Compute model predictions for test data
pred = model.predict(x_test)

actual = rcsi_test
r2_linear = stats.pearsonr(actual, pred)[0] ** 2
r2_linear

0.014429316077809751

In [37]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state=0,warm_start=True,max_depth=5,min_samples_split= 4)

# Fit estimator and display score
rfc = rfc.fit(x_train,rcsi_train)

# Regress on test data
pred = rfc.predict(x_test)

actual = rcsi_test
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

0.0033772910268296593

In [8]:
from sklearn.ensemble import ExtraTreesRegressor

# Create Regressor with default properties
extratree = ExtraTreesRegressor(random_state=0)

# Fit estimator and display score
extratree = extratree.fit(x_train, y_train[labels[1]])
# Regress on test data
pred = extratree.predict(x_test)

actual = y_test[labels[1]]
r2_et = stats.pearsonr(actual, pred)[0] ** 2
r2_et




0.0043088991745618933

In [9]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(warm_start=False,random_state =0, validation_fraction=0.1,learning_rate='adaptive')
# Fit estimator and display score
mlp = mlp.fit(x_train, y_train[labels[1]])
pred = mlp.predict(x_test)

actual = y_test[labels[1]]
r2_mlp= stats.pearsonr(actual, pred)[0] ** 2
r2_mlp

0.0080868793247701794

In [9]:
from sklearn.svm import SVR

# Create Regressor with default properties
svr = SVR(kernel='linear',degree =4)

# Fit estimator and display score
svr = svr.fit(x_train, y_train[labels[1]])
pred = svr.predict(x_test)

actual = y_test[labels[1]]
r2_svr= stats.pearsonr(actual, pred)[0] ** 2
r2_svr


0.0011961974509173241

In [ ]:
from sklearn.gaussian_process import  GaussianProcessRegressor

# Import our kernels
from sklearn.gaussian_process.kernels import Matern, WhiteKernel



# Define custom kernel (Matern + noise)
krnl = 2. + Matern(length_scale=1, nu=1.8) + WhiteKernel(noise_level=10)

# Create Regressor with specified properties
gpr = GaussianProcessRegressor(kernel=krnl, random_state=0)

# Fit estimator and display score
gpr= gpr.fit(x_train, y_train[labels[1]])
pred = gpr.predict(x_test)
actual = y_test[labels[1]]
r2_gpr= stats.pearsonr(actual, pred)[0] ** 2
r2_gpr



In [10]:

from sklearn.linear_model import Ridge

# Define different alpha values for different fits
alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge = Ridge(alpha=20,fit_intercept=False)

ridge.fit(x_train, y_train[labels[1]])
pred = ridge.predict(x_test)

actual = y_test[labels[1]]
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

0.010920801347841222

In [11]:
from sklearn.linear_model import BayesianRidge
bridge = BayesianRidge(compute_score=True)
bridge.fit(x_train, y_train[labels[1]])
pred = bridge.predict(x_test)

actual = y_test[labels[1]]
r2_bridge= stats.pearsonr(actual, pred)[0] ** 2
r2_bridge

0.010868246067988547

In [12]:
from sklearn.linear_model import Lasso
alpha = [1E-4, 1E-2, 0.1, 1.0]

ls = Lasso(alpha=alpha[2], fit_intercept=True)


ls = ls.fit(x_train, y_train[labels[1]])
pred = ls.predict(x_test)

actual = y_test[labels[1]]
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

0.010110781818608165

In [13]:
from sklearn.linear_model import ElasticNet
alpha = [1E-4, 1E-2, 0.1, 1.0]

en = ElasticNet(alpha=alpha[2], l1_ratio=0.5)


en.fit(x_train, y_train[labels[1]])
pred = en.predict(x_test)

actual = y_test[labels[1]]
r2_en= stats.pearsonr(actual, pred)[0] ** 2
r2_en


0.010173230252370151

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

# Create Regressor with default properties
gbr = GradientBoostingRegressor(random_state=0)

gbr.fit(x_train, y_train[labels[1]])
pred = gbr.predict(x_test)

actual = y_test[labels[1]]
r2_gbr= stats.pearsonr(actual, pred)[0] ** 2
r2_gbr


0.003212593484991158